## Payden McBee
## HW2: Bayes Optimal Classifiers
### Due: 1400 on 5 Oct 2020

<p style="color:blue"> 1. (40) The authors in [1] describe a marketing campaign by a bank in Portugal.
Modify the Exercise3.2LDAQDA IrisSoution python code to use the
data (bank-full.csv) from this marketing campaign with only the predictor
variables age, balance, and duration and the response variable, y, to
create the Bayes optimal classifiers for each of the conditions listed below.
Provide your python code for each of these cases.
  <br>  (a) Assume Gaussian class conditional likelihoods with unequal variancecovariance
matrices with each of the following additional assumptions
applied singularly to each decision rule in this class:
    <br> i. Equal class priors and equal costs for misclassification;

To start, I upload the libraries I need and read the "bank-full.csv" into a dataframe. Then, I pull out the columns of interest, age, balance, and duration, and the response variable, y. I rename the  response variable as class and save it to a csv file. I did this because LDA/QDA files take a .csv file as input and expect the response column to be labelled class.  

In [1]:
import pandas as pd
import numpy as np
import sys,os
from io import StringIO
import QDA
import LDA
bank = pd.read_csv("bank-full.csv")
bank_r = bank[['age', 'balance', 'duration','y']].copy()
bank_r.rename({'y': 'Class'}, axis=1, inplace=True)
#bank_r.to_csv('bank_r.csv')


a)i) Since I am assuming Gaussian class conditional likelihoods with unequal variancecovariance matrices, I use the QDA model. I use the QDA class as provided without altering it. Then, I create the uninformative_priors variable for the equal class priors and equal_costs to both be 1 for misclassification.

In [4]:
model_qda = QDA.QDA('bank_r.csv')
uninformative_priors = {
    "yes": 1 / 2,
    "no": 1 / 2
}
equal_costs = {
    "yes": 1,
    "no": 1
}

I then create the compute_probabilities and computeAccuracy functions as below, and then implement them to find the overall accuracy and the accuracy per class. The compute_probabilities function is called in the computeAccuracy function. 

In [5]:
 def compute_probabilities(self, x, priors):
        # compute and output the probability of each class and the maximum probability class

        # check that the input data x has the correct number of rows
        if not (len(x) == self.num_cols):
            print('Data vector has wrong number of values.')
            return -1

        # reformat x as a numpy array, incase the user input a list
        x = np.asarray(x)

        # first compute the likelihood of each class
        likelihoods = np.zeros(len(self.class_names))
        idx = 0
        for name in self.class_names:
            likelihoods[idx] = multivariate_gaussian_pdf(x, self.means[name], self.covs[name])
            idx = idx + 1

        # compute the probabilities of each class
        probabilites = np.zeros(len(self.class_names))
        idx = 0
        for name in self.class_names:
            probabilites[idx] = likelihoods[idx]*priors[name]#multivariate_gaussian_pdf(x, self.means[name], self.cov)
            idx = idx + 1
        probabilites = probabilites/probabilites.sum()
        return probabilities

In [8]:
 def computeAccuracy(self,priors,costsD):
        costs = np.zeros(self.class_names.shape)
        idx = 0
        for name in self.class_names:
            costs[idx] = costsD[name]
            idx=idx+1
        predClass = ['' for i in range(self.num_rows)]
        idx = 0
        for x in self.data:
            predClass[idx] = self.class_names[
                np.argmax(costs*self.compute_probabilities(x, priors), axis=0)]
            idx = idx + 1
        idx = 0
        accuracy = 0
        tp_PerClass = {classLabel: 0 for classLabel in self.data_labels}

        for y in self.data_labels:
            if y == predClass[idx]:
                accuracy = accuracy + 1
                tp_PerClass[y] = tp_PerClass[y] + 1
            idx = idx + 1
        accuracy = accuracy /  self.num_rows
        acc_PerClass = {label:tp_PerClass[label] / self.num_obs[label] for label in tp_PerClass}
        return accuracy, acc_PerClass

In [9]:
accuracy, acc_PerClass = \
model_qda.computeAccuracy(uninformative_priors,equal_costs)
print("QDA, uninformative priors, equal costs")
print(accuracy)
print(acc_PerClass)

QDA, uninformative priors, equal costs
0.8373183517285616
{'no': 0.8407394419117279, 'yes': 0.8114955568160332}


I computed the overall accuracy to be about 84% and the accuracy for no and yes to be 84% and 81%, respecively.   

1)a)ii) Next, I compute the accuracies for a prior of 0.9 for not selecting the new bank service and the same equal misclassification costs are before. I  use the same computeAccuracy function as above. 

In [10]:
informative_priors = {
    "yes": 1 / 10,
    "no": 9/10
}

accuracy, acc_PerClass = model_qda.computeAccuracy(informative_priors,equal_costs)
print("QDA, informative priors, equal costs")
print(accuracy)
print(acc_PerClass)

QDA, informative priors, equal costs
0.8852712835371923
{'no': 0.9609488502580031, 'yes': 0.3140480242011722}


Using the informative prior, the overall classification accuracy has risen to approximately 86%. However, when I inspect the per class accuracy, I see that the gains have come from the no class, rising to 96%. However, the correct classification for the yes class has dropped to 31.4%. This is due to class imbalance. Since the no class is much more likely, if we guess no more often, we're likely to have a higher accuracy. I show the class imbalance below, where the yes  response makes up 12% and the no response 88% of the total responses. 

In [32]:
print('yes',np.round(100*sum(model_qda.data_labels == 'yes')/model_qda.num_rows),'%')
print('no',np.round(100*sum(model_qda.data_labels == 'no')/model_qda.num_rows),'%')


yes 12.0 %
no 88.0 %


1)a)iii) I now compute the accuracy for a prior of 0.9 for not selecting the new bank service and the cost of classifying a customer as not a new service candidate when there are as 15 times the cost of classifying a customer as a new service customer when they are not. I run the computeAccuracy function as implemented above.

In [33]:
uneq_costs = {
    "yes" : 15,
    "no" : 1
}
accuracy, acc_PerClass = model_qda.computeAccuracy(informative_priors,uneq_costs)
print("QDA, informative priors, unequal costs")
print(accuracy)
print(acc_PerClass)

QDA, informative priors, unequal costs
0.7630665103625224
{'no': 0.7470567606833325, 'yes': 0.8839100018907166}


The overall classificaiton accuracy and the no classification accuracy went down to 76% and 75%, respectively. The accuracy for the yes response rose to 88%, the highest of all the 3 scenarios. 

1) b)i) Since I am assuming Gaussian class conditional likelihoods with equal variancecovariance matrices, I use the LDA model. I use the LDA class as provided without altering it. Then, I use the uninformative_priors variable for the equal class priors and equal_costs variable for misclassification, as descrived in Part (1i). I use create the compute_probabilities and computeAccuracy functions, as implemented above, to find the overall accuracy and the accuracy per class.

In [36]:
model_lda = LDA.LDA('bank_r.csv')
#part i
accuracy, acc_PerClass = model_lda.computeAccuracy(uninformative_priors,equal_costs)
print("LDA, uninformative priors, equal costs")
print('Overall Accuracy',np.round(100*accuracy),'%')
print(acc_PerClass)

LDA, uninformative priors, equal costs
Overall Accuracy 85.0 %
{'no': 0.851886178047192, 'yes': 0.7982605407449423}


The overall classification accuracy is 85%, and the accuracy for no and yes to be 85% and 85%, respecively. 

1)b)ii) Next, I compute the accuracies for a prior of 0.9 for not selecting the new bank service and the same equal misclassification costs are before. I  use the same computeAccuracy function as above. 

In [37]:
accuracy, acc_PerClass = model_lda.computeAccuracy(informative_priors,equal_costs)
print("LDA, informative priors, equal costs")
print('Overall Accuracy',np.round(100*accuracy),'%')
print(acc_PerClass)

LDA, informative priors, equal costs
Overall Accuracy 89.0 %
{'no': 0.9753769851209859, 'yes': 0.27982605407449423}


By using the informative priors, the overall accuracy and no response accuracy increases, as expected, though higher than in the case of LDA. Again, as expected, the accuracy for the yes class drops due to the class imbalance. 

1)b)iii I now compute the accuracy for a prior of 0.9 for not selecting the new bank service and the cost of classifying a customer as not a new service candidate when there are as 15 times the cost of classifying a customer as a new service customer when they are not. I run the computeAccuracy function as implemented above.

In [38]:
accuracy, acc_PerClass = model_lda.computeAccuracy(informative_priors,uneq_costs)
print("LDA, informative priors, unequal costs")
print(accuracy)
print(acc_PerClass)

LDA, informative priors, unequal costs
0.7814691114994139
{'no': 0.7660688342267421, 'yes': 0.8977122329362829}


The overall classificaiton accuracy and the no classification accuracy went down to 78% and 77%, respectively. The accuracy for the yes response rose to 89.8%, the highest of all the 3 scenarios. These accuracies are also higher than the QDA model, though not by much. It's possible that the QDA model overfit the data. 

<p style="color:blue"> 2) (25) Use numpy and pandas to develop a Naive Bayes classifier for edible mushrooms with the data in MushroomData.csv and MushroomVariablestxt. Use 2/3 of the observations to train the classifier and test on 1/3.

To begin, I read in the Mushroom.csv file into a pandas dataframe. I drop the last row, which is full of NaNs. I also read in the MushroomVariables.txt and set them as the column labels for the mushroom dataframe. I drop the class label, the first column, to create categories variable. This is a list of the feature variables. 

In [49]:
mushroom = pd.read_csv("MushroomData.csv",header=None)
mushroom.drop([mushroom.shape[0]-1],inplace=True) #get rid of NaN in last row
m_vars = pd.read_csv('MushroomVariables.txt', sep=",", header=None,dtype=str)
categories = m_vars.drop(columns=[0])
mushroom.columns = m_vars.values[0]

Next, I add the variable name to the word in each column. This is because some words are repeated in the different columns, but they do not carry the same meaning. For example, the color for the cap-color variable and for the veil-color may both be white, but they refer to different features. Furthermore, I keep the None's, since they are informative of the absense of a feature, such as the ring-number.   

In [50]:
Dictionary= []
for cat in categories:
    mushroom[categories[cat][0]]= mushroom[categories[cat][0]].apply(lambda word:word+categories[cat][0])
    Dictionary.append(mushroom[categories[cat][0]].unique().tolist())

I separate the mushroom dataframe into the edible and nonedible classes. Then I put 2/3 of each of the classes into the training set and 1/3 into the test set, thus having an equal balance of classes in the training and test sets. 

In [51]:
edible_df = mushroom[mushroom['edible_class'] == 'EDIBLE']
nonedible_df = mushroom[mushroom['edible_class'] != 'EDIBLE']
tng_ed_msk = np.random.rand(len(edible_df)) < 0.67
tst_ed_msk = ~tng_ed_msk
tng_ned_msk = np.random.rand(len(nonedible_df)) < 0.67
tst_ned_msk = ~tng_ned_msk
train_frames = [edible_df[tng_ed_msk], nonedible_df[tng_ned_msk]]
trainSet = pd.concat(train_frames)
test_frames = [edible_df[tst_ed_msk], nonedible_df[tst_ned_msk]]
testSet = pd.concat(test_frames)

I create a dictionary, prior, containing the number of each class' occurence. I also create the likelihood and evidence dictionaries that hold a probability for each word given the class. I add in a small term, alpha, to keep the likelihood of a word given a class from being equal to zero. I also normalize by the number of unique words in a feature multiplied by alpha. 

In [52]:
#training
prior = {label:trainSet[trainSet['edible_class'] == label].shape[0] for label in trainSet['edible_class'].unique()}
likelihood = dict.fromkeys([item for sublist in Dictionary for item in sublist],[0,0])
evidence = dict.fromkeys([item for sublist in Dictionary for item in sublist],0)
numTrngObs = trainSet.shape[0]
numEdTrngObs = sum(trainSet['edible_class'] == 'EDIBLE')
numNonedTrngObs = sum(trainSet['edible_class'] == 'POISONOUS')
alpha = 0.01
for cat in categories:
    for word in Dictionary[cat-1]:
        evidence[word] = sum(trainSet[categories[cat][0]] == word)/numTrngObs
        if sum(trainSet[categories[cat][0]] == word) >0:
            likelihood[word] =  ((sum((trainSet[categories[cat][0]] == word) & (trainSet['edible_class'] == 'EDIBLE'))+alpha)/(numEdTrngObs+alpha*len(Dictionary[cat-1])),(sum((trainSet[categories[cat][0]] == word) & (trainSet['edible_class'] == 'POISONOUS'))+alpha)/(numNonedTrngObs+alpha*len(Dictionary[cat-1])))

I create the prior probabilities for each class. 

In [ ]:
priorProbs = np.array([prior['EDIBLE']/numTrngObs,prior['POISONOUS']/numTrngObs])

I create a function to predict the label for each row given the features, likelihood, evidence and prior probabilities. 

In [53]:
def predict(dataSet,likelihood,evidence,priorProbs):
    numObs = dataSet.shape[0]
    predClass = ['' for i in range(numObs)]

    dataF = np.asarray(dataSet.drop('edible_class', axis=1, inplace=False))
    data_labels = dataSet.loc[:]['edible_class']

    class_names = np.unique(data_labels)

    numClasses = class_names.shape[0]
    x_idx = 0
    for x in dataF:
        probRow = np.zeros(shape=(dataF.shape[1], numClasses))
        w = 0
        for word in x:
            for label in range(numClasses):
                probRow[w, label] = likelihood[word][label] / evidence[word]
            w = w + 1
        predClass[x_idx] = class_names[np.argmax(np.prod(probRow, axis=0) * priorProbs)]
        x_idx = x_idx + 1
    return predClass

I use the predict function to predict the labels and computeAccuracies function to compare the predicted labels to the true labels. I first do this for the training dataset to gauge performance. 

In [58]:
predictionsTrain= predict(trainSet,likelihood,evidence,priorProbs) #priorProbs is  learned from training  set,  same for train and test
data_labels_train = trainSet.loc[:]['edible_class']

In [59]:
def computeAccuracies(data_labels,predClass,priorNum):
    accuracy = 0
    numObs = data_labels.shape[0]
    tp_PerClass = {classLabel: 0 for classLabel in data_labels}
    idx = 0
    for y in data_labels:
        if y == predClass[idx]:
            accuracy = accuracy + 1
            tp_PerClass[y] = tp_PerClass[y] + 1
        idx = idx + 1
    accuracy = accuracy / numObs
    acc_PerClass = {label: tp_PerClass[label] / priorNum[label] for label in tp_PerClass}
    return accuracy, acc_PerClass

In [60]:
accuracy, acc_PerClass=computeAccuracies(data_labels_train,predictionsTrain,prior)#for test make sure to use prior num in test set, not same as train set, same for data_labels
print(accuracy)
print(acc_PerClass)

0.9903982930298719
{'EDIBLE': 0.9949799196787149, 'POISONOUS': 0.9852048558421851}


The overall accuracy for the training set is high, at 99%. The accuracies for the EDIBLE and POINSONOUS classes are 99.5% and 98.5%, respectively. I then predict the label and compute the accuracies for the test set.

In [61]:
predictionsTest = predict(testSet,likelihood,evidence,priorProbs)

data_labels_test = testSet.loc[:]['edible_class']
Test_prior = {label:testSet[testSet['edible_class'] == label].shape[0] for label in testSet['edible_class'].unique()}
test_accuracy, test_acc_PerClass = computeAccuracies(data_labels_test,predictionsTest,Test_prior)
print(test_accuracy)
print(test_acc_PerClass)


0.9949856733524355
{'EDIBLE': 0.9966666666666667, 'POISONOUS': 0.9930340557275542}


The overall accuracy and accuracies for each class are above 99%. The model generalized well to the test set. 